In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib qt

import umap
from features import *
from my_io import *

c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
c:\Users\ferat\.conda\envs\ep\lib\site-packages\umap\distances.py:1086: NumbaDeprecationWarning: The '

## Read EEG data

In [2]:
files = list()

base_folder = fr'Z:\Animal\Fabien\DATA\KMR'
for subject_folder in os.listdir(base_folder):
    subject_folder = os.path.join(base_folder, subject_folder)
    if os.path.isdir(subject_folder):
        for day_folder in os.listdir(subject_folder):
            day_folder = os.path.join(subject_folder, day_folder)
            if os.path.isdir(day_folder):
                for file in os.listdir(day_folder):
                    if file.endswith('.sef'):
                        file = os.path.join(day_folder, file)
                        files.append(file)
files

['Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d11\\KMR1_d11_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d13\\KMR1_d13_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d15\\KMR1_d15_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d17\\KMR1_d17_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d19\\KMR1_d19_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d21\\KMR1_d21_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d23\\abd_raw.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d23\\KMR1_d23_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d23\\thre.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d25\\KMR1_d25_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d27\\KMR1_d27_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d29\\KMR1_d29_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d3\\KMR1_d3_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\KMR\\KMR1\\KMR1_d5\\KMR1_d5_Raw_DS.sef',
 'Z:\\Animal\\Fabien\\DATA\\K

In [3]:
all_features = []
for file in files[:2]:
    day = file.split("\\")[4]
    subject = file.split("\\")[3]
    features = []
    raw = read_file(file)
    raw = raw.pick('eeg')
    raw.filter(1., None)
    for channel in raw.ch_names:
        raw = raw.copy().pick(channel)

        events, events_id = mne.events_from_annotations(raw, chunk_duration=1)
        epochs = mne.Epochs(raw, events, event_id=events_id, tmin=0, tmax=1, baseline=None, on_missing='ignore')
        data = epochs.get_data()
        
        activity_features = activity(data)
        features.append(activity_features)
        
        mobility_features = mobility(data)
        features.append(mobility_features)
        
        complexity_features = complexity(data)
        features.append(complexity_features)
        
        time_features = extract_time_feat(data)
        features.append(time_features.reshape((time_features.shape[0],-1)))
        
        frequency_features = extract_freq_feat(data, sfreq=epochs.info['sfreq'])
        features.append(frequency_features.reshape((frequency_features.shape[0],-1)))
        
        information_features = extract_information_feat(data, sfreq=epochs.info['sfreq'])
        features.append(information_features.reshape((information_features.shape[0],-1)))
        
        dwt_features = extract_dwt_feat(data)
        features.append(dwt_features.reshape((dwt_features.shape[0],-1)))

        events = epochs.events[:,2].reshape(-1,1)
        features.append(events)
        
        days = np.array([day] * len(epochs)).reshape(-1,1)
        features.append(days)
        
        subjects = np.array([subject] * len(epochs)).reshape(-1,1)
        features.append(subjects)
        
        features = np.hstack(features)
        all_features.append(features)
    

df = pd.DataFrame(np.vstack(all_features))

Cannot read recording date from file...
year 0 is out of range
Creating RawArray with float64 data, n_channels=16, n_times=14403520
    Range : 0 ... 14403519 =      0.000 ...  3600.880 secs
Ready.
bad_epoch.mrk
bad_epoch_manual.mrk
bad_epoch_manual_bckup.mrk
bad_HF_Nz.mrk
bad_tracks.csv
mrk_bck.mrk
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 13201 samples (3.300 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    9.0s finished


Used Annotations descriptions: ['background', 'null']
Not setting metadata
43 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 43 events and 4001 original time points ...
2 bad epochs dropped
Cannot read recording date from file...
year 0 is out of range
Creating RawArray with float64 data, n_channels=16, n_times=15108480
    Range : 0 ... 15108479 =      0.000 ...  3777.120 secs
Ready.
bad_epoch.mrk
bad_epoch_manual.mrk
bad_epoch_manual_bckup.mrk
bad_HF_Nz.mrk
bad_tracks.csv
mrk_bck.mrk
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Filter length: 13201 samples (

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  16 out of  16 | elapsed:    9.1s finished


Used Annotations descriptions: ['background', 'null']
Not setting metadata
35 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 35 events and 4001 original time points ...
0 bad epochs dropped


In [12]:
all_features[0].shape

(41, 755)

In [4]:
df = pd.DataFrame(np.vstack(all_features))
df.add_prefix('col_')

col_0               col_1               col_2   
0    3487.499156371272  13342.419311421861  13214.431906169733  \
1    4942.433674767433  17798.096433283106  18251.005434776253   
2   1743.0644316110167   4745.458225362248   4832.890775646226   
3    4620.963009588115  15227.613710276848   17356.14055141678   
4    5200.590548757951  20164.478890507788  22337.377616655132   
..                 ...                 ...                 ...   
71   6189.492318530051   12671.16284017295  14066.775422345161   
72    6969.87942756884  18734.649782121644  20813.674935101873   
73  3825.8975360658455  11698.868193180475   12679.81577164442   
74   4552.964082220764  13125.626928359956  14023.766665423555   
75  3232.2396391321136   6961.902469961808   6705.338503513786   

                 col_3               col_4               col_5   
0    6930.842926310629   64748.79822224872  39031.476886102675  \
1   13301.403821600992   58437.56637838277   51540.18836602768   
2    5676.887870677577   68352.48886558507  58284.508501093274   
3    6361.541058699187   56557.87192492895   49245.79330532854   
4   10660.729437386193   61840.13618728273   33471.25419153575   
..                 ...                 ...                 ...   
71   9924.975079520718   50801.96152332116   60928.12438262533   
72    6575.87490536022   47747.85600445742   90478.34565523066   
73   6759.603092831907  40574.850769402605  27559.215640157363   
74   7774.905539293192   33772.31707147514   46854.13253371327   
75  10579.765744300104   90284.13694069863   64448.18348136683   

                 col_6               col_7               col_8   
0    6994.854917430535   6703.946613241357  1398.9915940592514  \
1    11670.91568915246   5544.067668200861  2985.4035208614605   
2    5939.670457559995    7860.43460625916  1461.6494341296925   
3    9306.174176863107   6484.332352258988  1897.3634968586475   
4    7628.138291441225   5723.817258161497  3007.7706934187754   
..                 ...                 ...                 ...   
71   9877.890422229993  7434.4407307280535  2007.1666093914491   
72   7580.465599175343   7026.370849603427    4812.45159100194   
73   6609.212027921744   6648.966197329037  1161.8795584070754   
74   5551.720175337172   6164.838842382773  3743.5412973745565   
75  5308.8431252092305  10407.839623245472   1329.866053274696   

                 col_9  ...               col_745              col_746   
0    3663.194922152851  ...    1.3568275212456236  0.26180623875903064  \
1   2671.6157145408642  ...   0.25562753078775247   0.2549574951250924   
2   1261.1637012995348  ...   0.12127917721604398   0.2677937191653243   
3   1985.4009641281727  ...    1.0861340327933204    0.266687541676287   
4   2584.7496317487544  ...    0.2776600436432317  0.24411437864858102   
..                 ...  ...                   ...                  ...   
71   2727.078803081716  ...   -0.2752816108875282  0.22866179388404603   
72  2061.0932654612775  ...    0.3738648151474808  0.26770937887854035   
73  1745.2120088256074  ...  -0.12467004520911029  0.23770739456149037   
74   1618.395104147705  ...    0.8775166538699932  0.26911520748601825   
75    2316.83583480299  ...    0.3410325972763206   0.2542809642186045   

                col_747             col_748               col_749   
0    2.6465051675535425  1715.0683531090401     0.733271776839058  \
1    0.6782871347072302   855.5835172760045    0.5337004204075743   
2   -0.5054998380391088   712.7485930740322   0.09027990456069626   
3    1.6437283200344406   880.4267391364602    0.0667564915333272   
4   -1.9160912332376339  1379.0073493558345  -0.27591690548305275   
..                  ...                 ...                   ...   
71  -2.3630295735935145  2036.9166218585694   -0.9558440863071896   
72   0.6604417873046162  1006.8522606770475   0.24292564721986945   
73   0.6438961117078844  1848.1213144094854  -0.09812106452187905   
74   2.2365197299441846  1617.5456111323508    0.5643691375

In [5]:
df.columns

RangeIndex(start=0, stop=755, step=1)

### Umap

In [49]:
fit = umap.UMAP()
data = df.values[:,:-4]
u = fit.fit_transform(data)

In [50]:
plt.scatter(u[:,0], u[:,1], c=df.values[:,-3].astype(int))

In [70]:
import umap.plot 
umap.plot.output_notebook()

hover_data = df.iloc[: , -2:]
hover_data.columns = ['day', 'subject']

p = umap.plot.interactive(fit, labels=df.values[:,-3].astype(int),
                          hover_data=hover_data,
                          theme='fire',
                          background='black')
umap.plot.show(p)

Loading BokehJS ...

In [58]:
p.to

Figure(id='1008', ...)

In [59]:
from bokeh.plotting import figure, output_file, save

In [71]:
save(p, 'test.html')

D:\Python\envs\epilespy\lib\site-packages\bokeh\io\saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
D:\Python\envs\epilespy\lib\site-packages\bokeh\io\saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")


'C:\\Users\\Victor\\Documents\\GitHub\\brainhack-epilepsy\\notebooks\\test.html'

In [72]:
umap.plot.connectivity(fit, show_points=True)

<AxesSubplot: >